In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, Flatten
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import os
from math import floor
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(12345)


Found GPU at: /device:GPU:0


Using TensorFlow backend.


# Due to computer issues, mine must include these code below

In [12]:


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
# If you want to train the model by yourself

# train data
with open('train_data.pickle', 'rb') as f:
    Data = pickle.load(f)
    Data = pd.DataFrame(Data)

numtrain = Data.shape[0]
numval = floor(numtrain*0.2)

# Train validation split
idx = random.sample(range(numtrain), numval) # 80% for training
all_index = list(np.setdiff1d(range(numtrain), idx))

# Getting X_train, X_val, y_train and scale them

X_train = Data.iloc[all_index,:].drop(2251, axis = 1)
X_val = Data.iloc[idx,:].drop(2251, axis = 1)
y_train = pd.get_dummies(Data.iloc[all_index,:][2251])

names = y_train.columns
mapping = {}
i = 0
for n in names:
    mapping[i] = n
    i+=1

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [5]:
X_train.shape

(11607, 2251)

In [9]:
# Construct CNN layers
# more para-LeNet-5
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (2251, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(186, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 2247, 6)           36        
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 1123, 6)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1123, 6)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1119, 16)          496       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 559, 16)           0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 559, 16)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 555, 32)           2592      
__________

In [10]:
# Fitting CNN model
numrest = numtrain-numval
cnn_model.fit(X_train.reshape(numrest, 2251, 1), y_train.values, batch_size = 128, epochs = 100, verbose = 1)

# Validating the CNN modela
pred = cnn_model.predict(X_val.reshape(numval, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)

# Create category mapping dictionary
y_val = Data.iloc[idx,:][2251]
    
# Calculating accuracy
predicted = np.vectorize(mapping.get)(out)
print('Accuracy: '+ str(round(np.sum(predicted == y_val)/numval,2) * 100) + '%')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
11607/11607 [==============================] - 3s 263us/step - loss: 3.4834 - acc: 0.5365
Epoch 2/100
11607/11607 [==============================] - 1s 120us/step - loss: 2.3963 - acc: 0.8999
Epoch 3/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.3403 - acc: 0.9181
Epoch 4/100
11607/11607 [==============================] - 1s 121us/step - loss: 2.3124 - acc: 0.9289
Epoch 5/100
11607/11607 [==============================] - 1s 120us/step - loss: 2.2975 - acc: 0.9332
Epoch 6/100
11607/11607 [==============================] - 1s 121us/step - loss: 2.2877 - acc: 0.9377
Epoch 7/100
11607/11607 [==============================] - 1s 128us/step - loss: 2.2858 - acc: 0.9388
Epoch 8/100
11607/11607 [==============================] - 1s 128us/step - loss: 2.2728 - acc: 0.9418
Epoch 9/100
11607/11607 [==============================] - 1s 120us/step - loss: 2.2657 - acc: 0.9453
Epoch 10/100
11607/11607 [========

11607/11607 [==============================] - 1s 121us/step - loss: 2.1954 - acc: 0.9895
Epoch 79/100
11607/11607 [==============================] - 1s 121us/step - loss: 2.1969 - acc: 0.9876
Epoch 80/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.1976 - acc: 0.9901
Epoch 81/100
11607/11607 [==============================] - 1s 123us/step - loss: 2.1954 - acc: 0.9891
Epoch 82/100
11607/11607 [==============================] - 1s 120us/step - loss: 2.1975 - acc: 0.9884
Epoch 83/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.1937 - acc: 0.9896
Epoch 84/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.1967 - acc: 0.9895
Epoch 85/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.1962 - acc: 0.9904
Epoch 86/100
11607/11607 [==============================] - 1s 121us/step - loss: 2.1944 - acc: 0.9889
Epoch 87/100
11607/11607 [==============================] - 1s 122us/step - loss: 2.19

In [6]:
#Or you can load our best model

from keras.models import load_model 
cnn_model= load_model('best_all-top3-except-two-zif8-top4.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
Test = pd.read_csv('test_data_14afterFFT.csv')

X_test = scaler.transform(Test.iloc[:,1:])
pred = cnn_model.predict(X_test.reshape(13, 2251, 1))
out = [np.argmax(p) for p in pred]
out = np.array(out)
predicted = np.vectorize(mapping.get)(out)

preds = []
for p in pred:
  P = []
  for i in np.argsort(p)[::-1][0:5]:
    P.append(mapping[i])
  preds.append(P)

for p in preds:
  print(p)

['JARMEU', 'JARMEU10', 'ZIF-67', 'ZIF-71', 'ZIF-65']
['JARMEU', 'JARMEU10', 'ZIF-67', 'ZIF-65', 'ZIF-71']
['JARMEU10', 'JARMEU', 'ZIF-67', 'ZIF-71', 'ZIF-8']
['ZIF-71', 'JARMEU10', 'AFEHUO', 'ZIF-9', 'MINVUA']
['ZIF-71', 'AFEHUO', 'JARMEU10', 'ZIF-9', 'MINVUA']
['ZIF-71', 'AFEHUO', 'JARMEU10', 'ZIF-9', 'MINVUA01']
['ZIF-90', 'DEFKUU', 'ZIF-8', 'AFIXES', 'ZIF-73']
['ZIF-90', 'AFIXES', 'DEFKUU', 'ZIF-8', 'ZIF-81']
['AFIXES', 'ZIF-8', 'DUT-23', 'MOF-5', 'ZIF-90']
['ZIF-90', 'AFIXES', 'ZIF-65', 'ZIF-8', 'DEFKUU']
['ZIF-90', 'AFIXES', 'DUT-23', 'JOSNAG', 'DEFKUU']
['ZIF-90', 'AFIXES', 'ZIF-65', 'DEFKUU', 'ZIF-8']
['ZIF-90', 'AFIXES', 'AFIXAO', 'DEFKUU', 'ZIF-71']
